In [2]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import mysql.connector
import natsort

In [3]:
transport = AIOHTTPTransport(url="https://graphql.anilist.co")
client = Client(transport=transport, fetch_schema_from_transport=True)

In [89]:
query = gql("""
query($page: Int, $perPage: Int, $numCharacters: Int) {
  Page(page: $page, perPage: $perPage) {
    media(sort: [POPULARITY_DESC]) {
      title {
        romaji(stylised: false)
      },
      studios(isMain: true) {
        nodes {
          name
        }
      },
      description(asHtml: true),
      duration,
      averageScore
      startDate {
        year,
        month,
        day
      },
      episodes,
      coverImage {
        extraLarge
      },
      trailer {
        id,
        site
      },
      
      streamingEpisodes {
        title,
        thumbnail
      }
      
      characters(sort: [RELEVANCE], page: 0, perPage: $numCharacters) {
        edges {
          node {
            name {
              full
            },
            image {
              large
            },
            description(asHtml: true)
          },
          voiceActors(sort: [RELEVANCE]) {
            name {
              full
            },
            image {
              large
            },
            description(asHtml: true)
          }
        }
      }
    }
  }
}
""")

params = {
    "page": 1,
    "perPage": 50,
    "numCharacters": 10
}

result = await client.execute_async(query, variable_values=params)
animes = result['Page']['media']

In [63]:
def ex(obj, path):
    props = path.split('.')
    for prop in props:
        if prop in obj:
            obj = obj[prop]
        else:
            return None
    return obj

def moveData(db):
    def datestr(a):
        return f"{a['year']:04d}-{a['month']:02d}-{a['day']:02d}"

    def trailer(b):
        if b == None:
            return None
        YT_FORMAT = "https://www.youtube.com/embed/{}"
        DM_FORMAT = "https://www.dailymotion.com/embed/video/{}"
        if (b['site'] == 'youtube'):
            return YT_FORMAT.format(b['id'])
        elif (b['site'] == 'dailymotion'):
            return DM_FORMAT.format(b['id'])
        else:
            return None

    def studio(b):
        nodes = ex(b, 'studios.nodes')
        if nodes != None and len(nodes) >= 1:
            return ex(nodes[0], 'name')
        return None

    cursor = db.cursor()
    add_anime = "INSERT INTO anime (nome, studio, trama, durata_episodi, voto_medio, data_rilascio, numero_episodi, immagine_copertina, trailer) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    add_personaggio = "INSERT INTO personaggio (id_anime, id_doppiatore, nome, descrizione, immagine) VALUES (%s, %s, %s, %s, %s)"
    add_episodio = "INSERT INTO episodio (id_anime, numero, titolo, thumbnail) VALUES (%s, %s, %s, %s)"
    add_doppiatore = "INSERT INTO doppiatore (nome, immagine, info) VALUES (%s, %s, %s)"
    episodes_data = []
    character_data = []
    for anime in animes:
        def t(s):
            return ex(anime, s)

        if studio(anime) == None:
            continue

        # Anime data
        anime_data = (t('title.romaji'), studio(anime), t('description'), t('duration'), t('averageScore'), datestr(t('startDate')), t('episodes'), t('coverImage.extraLarge'), trailer(t('trailer')))

        cursor.execute(add_anime, anime_data)
        anime_id = cursor.lastrowid

        # Episodes
        eps = natsort.natsorted(ex(anime, 'streamingEpisodes'), key=lambda x: x['title'])
        ep_num = 1
        for ep in eps:
            def t(s):
                return ex(ep, s)
            ep_data = (anime_id, ep_num, t('title'), t('thumbnail'))
            episodes_data.append(ep_data)
            ep_num += 1
        
        # Characters
        characters = ex(anime, 'characters.edges')
        for character in characters:
            info = ex(character, 'node')
            staff = ex(character, 'voiceActors')

            # Insert doppiatore se esiste
            id_doppiatore = None
            if len(staff) >= 1:
                dopp = staff[0]
                dopp_data = (ex(dopp, 'name.full'), ex(dopp, 'image.large'), ex(dopp, 'description'))
                cursor.execute(add_doppiatore, dopp_data)
                id_doppiatore = cursor.lastrowid

            # Add character
            char_data = (anime_id, id_doppiatore, ex(info, 'name.full'), ex(info, 'description'), ex(info, 'image.large'))
            character_data.append(char_data)

    cursor.executemany(add_episodio, episodes_data)
    cursor.executemany(add_personaggio, character_data)
    cursor.close()

In [90]:
db = mysql.connector.connect(
    host='master',
    user='root',
    password='PwNuova2!',
    database="uniproj"
)

try:
    moveData(db)
    db.commit()
finally:
    db.close()

IntegrityError: 1048 (23000): Column 'studio' cannot be null

In [ ]:
def ex(obj, path):
    props = path.split('.')
    for prop in props:
        if prop in obj:
            obj = obj[prop]
        else:
            return None
    return obj

def studio(b):
    nodes = ex(b, 'studios.nodes')
    if nodes != None and len(nodes) >= 1:
        return ex(nodes[0], 'name')
    return None

m = 0
for anime in animes:
    print(ex(anime, 'title.romaji'))
    print(studio(anime))
    print(anime['studios'])
    characters = ex(anime, 'characters.edges')
    for character in characters:
        info = ex(character, 'node')
        staff = ex(character, 'voiceActors')
        # if (len(ex(info, 'description')) == 19391):
        #     print(info)
        m = max(len(ex(info, 'description')), m)
        # if ex(info, 'title.full') == None:
        #     print(info)
print(m)